In [ ]:
import psycopg2
from psycopg2.errors import DuplicateColumn

conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

def get_data(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
    return data

def get_data_dict(sql):
    with conn.cursor() as cur:
        cur.execute(sql)
        data = cur.fetchall()
        col_name = [col.name for col in cur.description]
    return [ dict(zip(col_name,d)) for d in data]


def get_colname(table):
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM {} LIMIT 1;'.format(table))
        data = [col.name for col in cur.description]
    return data

def get_table_list():
    return [table[0] for table in get_data('SELECT relname FROM pg_stat_user_tables;')]

def get_sequence_list():
    return [seq[0] for seq in get_data('''SELECT c.relname FROM pg_class c LEFT join pg_user u ON c.relowner = u.usesysid WHERE c.relkind = 'S';''')]

def exec_transactions(sqls):
    with conn.cursor() as cur:
        for sql in sqls:
            cur.execute(sql)
    conn.commit()

In [ ]:
try:
    exec_transactions([
    '''ALTER TABLE participant ADD COLUMN after_id integer;''',
    '''ALTER TABLE participant ADD COLUMN training_id integer ''',
    '''ALTER TABLE participant ADD COLUMN origin varchar''',
    ])
except DuplicateColumn as err:
    print(err)
    conn = psycopg2.connect('user=honomara dbname=honomara password=honomara')

In [ ]:
exec_transactions(['''
UPDATE participant SET after_id = after.after_id, origin='after'
FROM after WHERE participant.id = after.id;
''',
'''
UPDATE participant SET training_id = training.training_id, origin='training'
FROM training WHERE participant.id = training.id;
'''])

In [ ]:
import mysql.connector
conn2 = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'honomara',
    password =  'honomara',
    database = 'honomara',
)

cur2 = conn2.cursor(dictionary=True)

In [ ]:
create_after_participants_table = '''
CREATE TABLE IF NOT EXISTS  after_participants (
    member_id INT NOT NULL,
    after_id INT NOT NULL
);
'''
create_training_participants_table='''
CREATE TABLE IF NOT EXISTS  training_participants (
    member_id INT NOT NULL,    
    training_id INT NOT NULL
);
'''

cur2.execute(create_after_participants_table)
cur2.execute(create_training_participants_table)
cur2.execute('''ALTER TABLE after_participants ADD CONSTRAINT FOREIGN KEY (member_id) REFERENCES members(member_id);''')
cur2.execute('''ALTER TABLE after_participants ADD CONSTRAINT  FOREIGN KEY (after_id) REFERENCES afters(after_id);''')
cur2.execute('''ALTER TABLE training_participants ADD CONSTRAINT  FOREIGN KEY (member_id) REFERENCES members(member_id);''')
cur2.execute('''ALTER TABLE training_participants ADD CONSTRAINT  FOREIGN KEY (training_id) REFERENCES trainings(training_id);''')
conn2.commit()


In [ ]:
get_data_dict('SELECT * FROM participant LIMIT 1;')

In [ ]:
conn2.commit()

In [ ]:
for row in get_data_dict('SELECT * FROM participant;'):
    if row['origin'] == 'after':
        cur2.execute('INSERT INTO after_participants (member_id,after_id) VALUES (%s,%s);',(row['person_id'],row['after_id']))
    elif row['origin'] == 'training':
        cur2.execute('INSERT INTO training_participants (member_id,training_id) VALUES (%s,%s);',(row['person_id'],row['training_id']))
conn2.commit()

## miscellaneous info

In [ ]:
get_table_list()

In [ ]:
get_data('SELECT * FROM participant WHERE origin IS NULL LIMIT 10;')

In [ ]:
cur2.execute('SHOW TABLES;')
cur2.fetchall()

In [ ]:
get_data('SELECT origin,count(id) FROM participant GROUP BY origin;')

In [ ]:
cur2.execute('DESCRIBE trainings ;')
cur2.fetchall()

In [ ]:
cur2.execute('DESCRIBE members ;')
cur2.fetchall()

In [ ]:
cur2.execute('show engines;')
cur2.fetchall()

In [ ]:
cur2.execute('''use information_schema;''')
cur2.execute('''select table_schema, table_name, engine from tables WHERE table_schema='honomara';''')
cur2.fetchall()

In [ ]:
cur2.execute('DESCRIBE trainings_participants;')
cur2.fetchall()